In [19]:
import numpy
import scipy.special # Contains the sigmoid function which is named expit(x) in the library
import matplotlib.pyplot
%matplotlib inline

# Neural Network class definition
class NeuralNetwork:    
    #Constructor
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        # Number of nodes in each layer
        self.iNodes = inputNodes
        self.hNodes = hiddenNodes
        self.oNodes = outputNodes
        # learning rate
        self.lRate = learningRate
        #Create Weight Matrixes
        self.w_input_hidden = numpy.random.normal(0.0, pow(self.iNodes, -0.5),(self.hNodes, self.iNodes))
        self.w_hidden_output = numpy.random.normal(0.0, pow(self.hNodes, -0.5),(self.oNodes, self.hNodes))
        #activation function AKA Sigmoid function
        # lambda keyword declares the creation of a lambda function
        # assigning a lambda function creates something similar to a c# delegate or a function pointer
        self.activation_function = lambda x: scipy.special.expit(x) 
        pass
    
    #training function for network
    def Train(self, inputs_list, targets_list):
        # Converts target lists into 2D arrays
        targets = numpy.array(targets_list, ndmin = 2).T
        # Obtains ouput matrix
        final_outputs, hidden_outputs, inputs = self.CalculateOutput(inputs_list)
        # Error (target - actual)
        output_errors = targets - final_outputs
        # Calculation for the error of the hidden layer
        hidden_errors = numpy.dot(self.w_hidden_output.T, output_errors)
        # Update the links between the output layer and the hidden layer
        self.w_hidden_output += (self.lRate * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs)))
        # Update the links between the hidden layer and the input layer
        self.w_input_hidden += (self.lRate * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs)))
        pass
    
    #query the network
    def Query(self, inputs_list):
        return self.CalculateOutput(inputs_list)[0]
        pass
    
       #Calculates the final outputs from a given input
    def CalculateOutput(self, inputs_list):
        # Converts inputs list into a 2D array
        inputs = numpy.array(inputs_list, ndmin = 2).T
        # Combines the inputs with the input to hidden layer weights
        hidden_inputs = numpy.dot(self.w_input_hidden, inputs)
        # Applies the sigmoid fuction to the weighted input
        hidden_outputs = self.activation_function(hidden_inputs)
        # Combines the inputs with the hidden to output layer weights
        final_inputs = numpy.dot(self.w_hidden_output, hidden_outputs)
        # Applies the sigmoid fuction to the weighted hidden layer outputs
        final_outputs = self.activation_function(final_inputs)       
        return (final_outputs, hidden_outputs, inputs)
        pass

In [76]:
# Setting the hyperparameters
epochs = 7
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.1
# Initializing the Network
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [77]:
# Retrieving the data
data_file = open("mnist_dataset/mnist_train.csv", 'r')
data_list = data_file.readlines()
data_file.close()

In [80]:
# Training Loop
for e in range(epochs):
    for record in data_list:
        #Split the record by the',' commas
        values = record.split(',')
        # Set the inputs to be values between .01 and .99 
        inputs = (numpy.asfarray(values[1:]) / 255.0 * 0.99) + 0.01
        # Create the target output values, the correct label will be .99, all others .01
        targets = numpy.zeros(output_nodes) + 0.01
        # values[0] is the target label for the current record
        targets[int(values[0])] = 0.99
        #
        n.Train(inputs,targets)
        pass
    pass

In [81]:
# Loading test Data
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [83]:
# Testing the Neural Network
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
   # print(correct_label, "Correct Label")
    inputs = (numpy.asfarray(all_values[1:])/ 255.0 * 0.99) + 0.01
    outputs = n.Query(inputs)
    label = numpy.argmax(outputs)
    #print(label, "Networks Answer")
    if(label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
    pass

In [84]:
# Print the Networks Performance on Test
scorecard_array = numpy.asarray(scorecard)
print("Performance =", scorecard_array.sum() / scorecard_array.size)

Performance = 0.9657
